<h1>A pharmacy in Milan: where to start</h1>
<h2>A data analysis for finding the best place in Milan for a new pharmacy business.</h2>
<h4><em>Applied Data Science Capstone by IBM/Coursera</em></h4>

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to analyze some data for finding the **best neighborhood in Milan** for starting a new **pharmacy** business. In this 1 million and 300 thousand people city there are indeed more than with 400 pharmacies and we have to struggle with multiple factors when trying to understand which of the 88 neighborhoods would be nice for a new one. Thus, **if you want to open a pharmacy in Milan** or if you just want to better understand this business situation in the city, this analysis is for you.
We will go through three important factors for reaching our aim.
* For gaining as many customers as possible we need a place with few competitors: which neighborhoods have got the **least number of pharmacies**?
* For reaching a high number of customers we need to serve a highly populated area: which neighborhoods have got the **highest population**?
* And finally, for being efficient we could need a place with some pharmaceutical wholesaler: which neighborhoods have got the **highest number of pharmaceutical distributors**? 

These three questions will guide our analysis and will help to find the right data to analyze for our goal. We will use data science tools for answering the initial question: **where is the best place in Milan for starting a new pharmacy business?**


## Data <a name="data"></a>

Basing our analysis on the three previous questions, we now need to find three key data:
* Number of pharmacies for every neighborhood
* Population of every neighborhood
* Number of pharmaceutical wholesalers for every neighborhood

All the information on neighborhood ID, coordinates, population and more, can be found on the official portal of **Milan municipality** (http://dati.comune.milano.it/), a site with many **open data-sets** about the city. There is also a nice georeferenced dataset with all the information about pharmaceutical distributors around the city.
We will then find the number of pharmacies thanks to **Foursquare API**, sending some request for a specific venue category (pharmacy) near a spot (center of neighborhood). 
All these datasets need to be **pre-processed** for fitting our needs. We are going to use **Python** libraries for this aim and **Excel** for fast preparing some tables. All data will be then loaded on a public url.

*Please, note that Milan neighborhoods are called NIL (Nuclei di Idendità Locale, that means Local Identity Nuclea).*


In [4]:
#Import all Python necessary libraries for data acquisition and processing
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import json
import requests 
from pandas.io.json import json_normalize 

### Pharmaceutical wholesaler distributors

Let's start reading the pharmaceutical distributors CSV dataset by Milan Municipality and filling a Pandas dataframe with it. Note that i uploaded the dataset on a personal shared url. 

In [5]:
distributors = pd.read_csv('http://asvalon.altervista.org/wp-content/uploads/2020/05/distributori-di-farmaci_v3_geo.csv', sep=';')
distributors.head()

,CODICEIDENTIFICATIVOSITO,DENOMINAZIONESITOLOGISTICO,INDIRIZZO,PARTITAIVA,CAP,CODICECOMUNEISTAT,DESCRIZIONECOMUNE,CODICEPROVINCIAISTAT,SIGLAPROVINCIA,DESCRIZIONEPROVINCIA,...,DESCRIZIONEDISTRIBUTORE,LOCALIZE,NUMEROCOMPLETO,CODICE_VIA,MUNICIPIO,ID_NIL,NIL,LONG_WGS84,LAT_WGS84,Location
0,18,Stabilimento di Milano,via matteo civitali 1,748210150,20148,15146,Milano,15,MI,Milano,...,Produttore,1,1,6569.0,7.0,57.0,SELINUNTE,9.136590,45.468945,"45.4689449631, 9.13658985457"
1,27,Lofarma S.p.A.,"viale cassala, 40",713510154,20143,15146,Milano,15,MI,Milano,...,Produttore,1,40,5275.0,6.0,44.0,NAVIGLI,9.165970,45.444011,"45.4440112757, 9.16597016441"
2,28,Mipharm S.p.A.,"via b. quaranta, 12",12304990158,20141,15146,Milano,15,MI,Milano,...,Produttore,1,NaN,NaN,5.0,38.0,RIPAMONTI,9.204009,45.436331,"45.4363314, 9.2040092"
3,30,SCHWARZ PHARMA S.p.A.,"via gadames, snc",7254500155,20151,15146,Milano,15,MI,Milano,...,Produttore,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59,LA COMMERCIALE FARMACEUTICA srl,via desenzano 6/a,55560775,20146,15146,Milano,15,MI,Milano,...,Grossista,1,6A,6610.0,7.0,52.0,BANDE NERE,9.140103,45.462419,"45.4624185522, 9.14010264607"


Don't worry about Italian, cos we are going to clean a lot of unnecessary data from this set and make it more clear and understandable.
Note that important data are on the columns:
* DENOMINAZIONESITOLOGISTICO, that is the name of the distributor
* ID_NIL, that is the code of the neighborhood
* NIL, that is the name of the neighborhood
* LONG_WGS84, the longitude of the site
* LAT_WGS84, the latitude of the site

Now let's clean the dataset and rearrange columns. Some rows are missing location data: we are dropping them.

In [7]:
distributors = distributors[['DENOMINAZIONESITOLOGISTICO','ID_NIL','NIL','LONG_WGS84','LAT_WGS84']]
distributors = distributors.dropna()
distributors = distributors[['DENOMINAZIONESITOLOGISTICO','ID_NIL','NIL','LAT_WGS84','LONG_WGS84']]
distributors.head()

,DENOMINAZIONESITOLOGISTICO,ID_NIL,NIL,LAT_WGS84,LONG_WGS84
0,Stabilimento di Milano,57.0,SELINUNTE,45.468945,9.136590
1,Lofarma S.p.A.,44.0,NAVIGLI,45.444011,9.165970
2,Mipharm S.p.A.,38.0,RIPAMONTI,45.436331,9.204009
4,LA COMMERCIALE FARMACEUTICA srl,52.0,BANDE NERE,45.462419,9.140103
5,V.I.M. srl,52.0,BANDE NERE,45.462419,9.140103


Now, let's visualize the location site of all the distributors with Python Folium library. Click on the marker for the name.

In [8]:
#Define coordinates of Milan
milan_coords = [45.4668, 9.1905]

#Create the map of Milan
milan_map_distributors = folium.Map(location = milan_coords, zoom_start = 12)

#Ad markers to the map for pharmaceutical wholesaler distributors in Milan
for i in range (0,72):
    dist_loc = distributors.iloc[i,3:5] #location of distributor
    dist_name = distributors.iloc[i,0] #name of distributor 
    folium.Marker(dist_loc, popup = dist_name).add_to(milan_map_distributors)
#Display the map
milan_map_distributors

Now we need to set the ID_NIL code as an integer type variable.

In [9]:
distributors['ID_NIL'] = distributors['ID_NIL'].astype(int)
distributors.head()

,DENOMINAZIONESITOLOGISTICO,ID_NIL,NIL,LAT_WGS84,LONG_WGS84
0,Stabilimento di Milano,57,SELINUNTE,45.468945,9.136590
1,Lofarma S.p.A.,44,NAVIGLI,45.444011,9.165970
2,Mipharm S.p.A.,38,RIPAMONTI,45.436331,9.204009
4,LA COMMERCIALE FARMACEUTICA srl,52,BANDE NERE,45.462419,9.140103
5,V.I.M. srl,52,BANDE NERE,45.462419,9.140103


Time to count the number of distributors for every neighborhood. Note that we represent the neighborhood with the ID_NIL code and that not every neighborhood has got some wholesaler.

In [10]:
dist_freq = distributors['ID_NIL'].value_counts()
dist_freq

13    9
79    7
30    6
21    6
15    4
68    3
52    3
48    3
80    2
38    2
28    2
20    2
26    2
44    2
12    2
22    1
18    1
16    1
11    1
19    1
33    1
23    1
35    1
36    1
51    1
57    1
60    1
61    1
67    1
70    1
75    1
10    1
Name: ID_NIL, dtype: int64

We then fill an excel file with the obtained data. In particular we are using an excel file that you can find on Milan Municipality portal, with information about every neighborhoods population. We simply add a column with the number of distributors. 

### Population

We now have an excel file with:
* ID_NIL and NIL name, for identifying every neighborhood
* Latitude and Longitude of the neighborhood
* dist_freq, that is the number of pharmaceutical distributors in the neighborhood
* pop, that is the population of the neighborhood

Let's read it and fill a Pandas dataframe.

In [41]:
nil_data = pd.read_excel('http://asvalon.altervista.org/wp-content/uploads/2020/05/NIL_coo.xlsx')
nil_data.head()

,ID_NIL,NIL,Latitude,Longitude,dist_freq,pop
0,17,Adriano,45.514271,9.245198,0,16947
1,80,Affori,45.513926,9.173530,2,24893
2,55,Baggio,45.462387,9.086922,0,29661
3,52,Bande_Nere,45.461504,9.136485,3,44230
4,46,Barona,45.438652,9.158203,0,17018


Let's now visualiza a map with neighborhood boarder and names. We are using a choropleth map build with a geojson file from gihub with Milan neighborood. The intensity of the color is based on the number of distributors in that neighborhood.

In [18]:
#Define coordinates of Milan
milan_coords = [45.4668, 9.1905]

#Create the map of Milan
milan_map_nil = folium.Map(location = milan_coords, zoom_start = 12)

#Ad markers to the map for pharmaceutical wholesaler distributors in Milan
for i in range (0,88):
    nil_loc = nil_data.iloc[i,2:4] #location of nil
    nil_name = nil_data.iloc[i,1] #name of nil
    folium.Marker(nil_loc, popup = nil_name).add_to(milan_map_nil)
    
milan_geo = r'https://raw.githubusercontent.com/abahgat/opendata-milano/master/geojson/quartieri.geojson'

milan_map_nil.choropleth(
    geo_data=milan_geo,
    data=nil_data,
    columns=['ID_NIL','dist_freq'],
    key_on='feature.properties.ID_NIL',
    fill_color='YlOrRd'
)
#Display the map
milan_map_nil


We are now going to visualize a choropleth map where the intensity represents the population of the neighborhood.

In [19]:
milan_map_pop = folium.Map(location = milan_coords, zoom_start = 12)
milan_map_pop.choropleth(
    geo_data=milan_geo,
    data=nil_data,
    columns=['ID_NIL','pop'],
    key_on='feature.properties.ID_NIL',
    fill_color='OrRd'
)
# display map
milan_map_pop

### Pharmacies number

It's now time for calling Foursquare API and finding how many pharmacies are located near every neighborhood.  
Let's start reorganizing our dataframe sorting it by ID_NIL code.

In [42]:
nil_data = nil_data.sort_values('ID_NIL')
nil_data.head()

,ID_NIL,NIL,Latitude,Longitude,dist_freq,pop
20,1,Duomo,45.464181,9.189738,0,16882
8,2,Brera,45.471236,9.187407,0,18199
29,3,Giardini_Porta_Venezia,45.474424,9.199361,0,38
32,4,Guastalla,45.459333,9.202480,0,15050
84,5,Vigentina,45.431374,9.187316,0,41216


We are going to send search requests to Foursquare, looking for pharmacy venues near in the radius of 800 meters near every neighborhood. Note that we are not interested in all the data contained in the response, but just in the number of found venues.   
We put the results into a CSV file and we save it, so we won't have to call Foursquare API every time we run the program.

In [23]:
# @hidden_cell

CLIENT_ID = '0HAQQIPDFB2F2PIABTVKJ3ANW4BETV4QE3Y3YTOMD4S4AB5P' # your Foursquare ID
CLIENT_SECRET = 'YZ4BCBZ2W0IO14MQQWWFEQKW32FNJVYJ1XJCOCSCPBPNDSGT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [24]:
import json
from pandas.io.json import json_normalize

pharma_num = pd.DataFrame(columns=['ID_NIL','NIL','pharma_num'])

LIMIT = 100
intent = 'checkin'
radius = 800
categoryId = '4bf58dd8d48988d10f951735'

for i in range(0,88):
    
    nil_latitude = nil_data.iloc[i,2]
    nil_longitude = nil_data.iloc[i,3]
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&intent={}&radius={}&limit={}&categoryId={},'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        nil_latitude, 
        nil_longitude,
        intent,
        radius, 
        LIMIT,
        categoryId)
    results = requests.get(url).json()["response"]['venues']
    df = pd.DataFrame(results)
    pharma_num = pharma_num.append({'ID_NIL':nil_data.iloc[i,0],'NIL':nil_data.iloc[i,1],'pharma_num':len(df.index)},ignore_index=True)
print(pharma_num)  

   ID_NIL                     NIL pharma_num
0       1                   Duomo         42
1       2                   Brera         29
2       3  Giardini_Porta_Venezia         24
3       4               Guastalla         26
4       5               Vigentina          6
..    ...                     ...        ...
83     84              Parco_Nord          2
84     85     Parco_delle_Abbazie          0
85     86       Parco_dei_Navigli          0
86     87      Parco_Agricolo_Sud          0
87     88    Parco_Bosco_in_città          0

[88 rows x 3 columns]


### Final dataset

We now have got every data we need and we put all into a dataframe where we have 
* neighborhoods data
* number of distributor
* population
* number of pharmacies

In [48]:
pharma_num_dropped = pharma_num.drop('NIL',axis=1)
final_data = pd.merge(left=nil_data, right=pharma_num_dropped, left_on='ID_NIL', right_on='ID_NIL')
final_data.head()

,ID_NIL,NIL,Latitude,Longitude,dist_freq,pop,pharma_num
0,1,Duomo,45.464181,9.189738,0,16882,42
1,2,Brera,45.471236,9.187407,0,18199,29
2,3,Giardini_Porta_Venezia,45.474424,9.199361,0,38,24
3,4,Guastalla,45.459333,9.202480,0,15050,26
4,5,Vigentina,45.431374,9.187316,0,41216,6
